## 1. Modelo dinámico de motor rotativo

In [1]:
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib.pyplot as plt

def animate_motor(results):
    positions = np.radians(results["position_unwrapped_deg"])
    throttle = results["throttle"]
    velocity = results["velocity_deg_s"]  # o est_vel, como prefieras
    time = results["time"]

    fig = plt.figure(figsize=(12, 8))

    # Polar para la orientación
    ax_polar = plt.subplot2grid((2, 2), (0, 0), rowspan=2, projection='polar')
    arrow, = ax_polar.plot([], [], lw=3)
    ax_polar.set_ylim(0, 1)
    ax_polar.set_title("Motor position (polar)")

    # Throttle
    ax_throttle = plt.subplot2grid((2, 2), (0, 1))
    throttle_line, = ax_throttle.plot([], [], lw=2)
    ax_throttle.set_xlim(time[0], time[-1])
    ax_throttle.set_ylim(0, 1.1)
    ax_throttle.set_title("Throttle")
    ax_throttle.set_ylabel("Throttle")

    # Velocity
    ax_velocity = plt.subplot2grid((2, 2), (1, 1))
    velocity_line, = ax_velocity.plot([], [], lw=2)
    ax_velocity.set_xlim(time[0], time[-1])
    ax_velocity.set_ylim(min(velocity)*1.1, max(velocity)*1.1)
    ax_velocity.set_title("Velocity")
    ax_velocity.set_xlabel("Time (s)")
    ax_velocity.set_ylabel("Velocity (deg/s)")

    def init():
        throttle_line.set_data([], [])
        velocity_line.set_data([], [])
        return arrow, throttle_line, velocity_line

    def update(frame):
        # Polar
        angle = positions[frame] % (2 * np.pi)
        arrow.set_data([angle, angle], [0, 1])

        # Throttle
        throttle_line.set_data(time[:frame], throttle[:frame])

        # Velocity
        velocity_line.set_data(time[:frame], velocity[:frame])

        return arrow, throttle_line, velocity_line

    ani = FuncAnimation(fig, update, frames=len(positions),
                        init_func=init, blit=True, interval=50)

    plt.tight_layout()
    plt.close(fig)
    return HTML(ani.to_html5_video())

animate_motor(results)

NameError: name 'results' is not defined

## 2. Modelo de encoder con ruido

In [ ]:
motor = RotaryMotor(
    inertia=0.02,
    friction_viscous=0.02,
    max_torque=0.1,
    friction_coulomb=0.02,
    external_torque=0.01
)
encoder = Encoder(motor, noise_std_deg=0.5)

# Simula un paso
motor.set_throttle(0.8)
motor.update(0.01)

print("True pos:", motor.get_position_deg())
print("Encoder pos:", encoder.read_position_deg())

In [ ]:
def simulate_motor_with_encoder():
    motor = RotaryMotor(
        inertia=0.02,
        friction_viscous=0.02,
        max_torque=0.1,
        friction_coulomb=0.02,
        external_torque=0.01
    )
    encoder = Encoder(motor, noise_std_deg=0.5)

    dt = 0.01
    t_max = 10.0
    steps = int(t_max / dt)

    times, throttle, pos_true, pos_true_unwrapped, pos_encoder = [], [], [], [], []

    for step in range(steps):
        t = step * dt

        if t < 3.0:
            th = t / 3.0
        elif t < 6.0:
            th = 1.0
        elif t < 8.0:
            th = 1.0 - (t - 6.0) / 2.0
        else:
            th = 0.0

        motor.set_throttle(th)
        motor.update(dt)

        times.append(t)
        throttle.append(th)
        pos_true.append(motor.get_position_deg())
        pos_true_unwrapped.append(motor.get_position_unwrapped_deg())
        pos_encoder.append(encoder.read_position_deg())

    return {
        "time": times,
        "throttle": throttle,
        "pos_true": pos_true,
        "pos_true_unwrapped": pos_true_unwrapped,
        "pos_encoder": pos_encoder
    }


# -----------------------------
# Reporte: posición real vs encoder
# -----------------------------

def plot_encoder_vs_true(results):
    fig, axes = plt.subplots(2, 1, figsize=(22, 7), sharex=True)

    axes[0].plot(results["time"], results["pos_true"], label="True position (wrapped)")
    axes[0].plot(results["time"], results["pos_encoder"], label="Encoder reading", alpha=0.7)
    axes[0].set_ylabel("Position (deg)")
    axes[0].set_title("Wrapped position: True vs Encoder")
    axes[0].grid()
    axes[0].legend()

    axes[1].plot(results["time"], results["pos_true_unwrapped"], label="True position (unwrapped)")
    axes[1].set_ylabel("Position (deg)")
    axes[1].set_xlabel("Time (s)")
    axes[1].set_title("True position (unwrapped)")
    axes[1].grid()
    axes[1].legend()

    plt.tight_layout()
    plt.show()


# -----------------------------
# Ejecutar todo
# -----------------------------

results = simulate_motor_with_encoder()
plot_encoder_vs_true(results)

## 3. Control de velocidad

In [ ]:
from collections import deque

class VelocityFilter:
    def __init__(self, window_size=5):
        """
        Filtro de media móvil.
        window_size: número de muestras a promediar.
        """
        self.history = deque(maxlen=window_size)

    def filter(self, value):
        self.history.append(value)
        return sum(self.history) / len(self.history)


In [ ]:
class ExponentialVelocityFilter:
    def __init__(self, alpha=0.2):
        self.alpha = alpha
        self.last = None

    def filter(self, value):
        if self.last is None:
            self.last = value
        else:
            self.last = self.alpha * value + (1 - self.alpha) * self.last
        return self.last


In [ ]:
class SpeedControllerPID:
    def __init__(self, motor, encoder, velocity_filter=None,
                 kp=0.2, ki=0.03, kd=0.02,
                 integral_limit=50.0,
                 deadband=1.0):
        """
        deadband: zona muerta (en deg/s) para ignorar errores minúsculos.
        """
        self.motor = motor
        self.encoder = encoder
        self.velocity_filter = velocity_filter

        self.kp = kp
        self.ki = ki
        self.kd = kd
        self.integral_limit = integral_limit
        self.deadband = deadband

        self.integral_error = 0.0
        self.prev_error = 0.0
        self.prev_pos = None
        self.prev_derivative = 0.0  # para suavizar derivada

    def estimate_velocity_deg_s(self, dt):
        pos = self.encoder.read_position_deg()

        if self.prev_pos is None:
            self.prev_pos = pos
            return 0.0

        delta = pos - self.prev_pos
        if delta > 180.0:
            delta -= 360.0
        elif delta < -180.0:
            delta += 360.0

        raw_velocity = delta / dt
        self.prev_pos = pos

        if self.velocity_filter:
            return self.velocity_filter.filter(raw_velocity)
        else:
            return raw_velocity

    def update(self, desired_velocity_deg_s, dt):
        measured_velocity = self.estimate_velocity_deg_s(dt)
        error = desired_velocity_deg_s - measured_velocity

        # ---------- DEADZONE ----------
        if abs(error) < self.deadband:
            error = 0.0

        # ---------- INTEGRAL ----------
        self.integral_error += error * dt
        self.integral_error = max(min(self.integral_error, self.integral_limit),
                                  -self.integral_limit)

        # ---------- DERIVATIVE ----------
        raw_derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        # Derivative smoothing
        alpha_d = 0.2
        derivative_error = (1 - alpha_d) * self.prev_derivative + alpha_d * raw_derivative

        output = (self.kp * error +
                  self.ki * self.integral_error +
                  self.kd * derivative_error)

        throttle = max(0.0, min(output, 1.0))
        self.motor.set_throttle(throttle)

        self.prev_error = error
        self.prev_derivative = derivative_error

        return {
            "measured_velocity": measured_velocity,
            "error": error,
            "throttle": throttle
        }

In [ ]:
def simulate_pid_control():
    # Componentes
    motor = RotaryMotor(
        inertia=0.02,
        friction_viscous=0.05,   # Sube un poco para amortiguar
        max_torque=1.6,
        friction_coulomb=0.02,
        external_torque=0.01
    )
    encoder = Encoder(motor, noise_std_deg=0.5)
    velocity_filter = ExponentialVelocityFilter(alpha=0.5)
    
    controller = SpeedControllerPID(
        motor,
        encoder,
        velocity_filter=velocity_filter,
        kp=0.1, ki=0.003, kd=0.002,
        integral_limit=30.0,
        deadband=1.0
    )

    dt = 0.01
    t_max = 10.0
    steps = int(t_max / dt)

    time, ref_vel, real_vel, est_vel, throttle = [], [], [], [], []
    errors, integrals, derivatives = [], [], []

    for step in range(steps):
        t = step * dt

        if t < 3.0:
            vel_ref = 30.0 * (t / 3.0)
        elif t < 6.0:
            vel_ref = 30.0
        elif t < 8.0:
            vel_ref = 30.0 * (1 - (t - 6.0) / 2.0)
        else:
            vel_ref = 0.0

        status = controller.update(vel_ref, dt)
        motor.update(dt)

        time.append(t)
        ref_vel.append(vel_ref)
        real_vel.append(motor.get_velocity_deg_s())
        est_vel.append(status["measured_velocity"])
        throttle.append(status["throttle"])

        errors.append(controller.prev_error)
        integrals.append(controller.integral_error)
        derivatives.append(controller.prev_derivative)

    return {
        "time": time,
        "ref_vel": ref_vel,
        "real_vel": real_vel,
        "est_vel": est_vel,
        "throttle": throttle,
        "errors": errors,
        "integrals": integrals,
        "derivatives": derivatives
    }

def plot_pid_terms(results):
    fig, axes = plt.subplots(3, 1, figsize=(22, 8), sharex=True)

    axes[0].plot(results["time"], results["errors"])
    axes[0].set_ylabel("Error (deg/s)")
    axes[0].set_title("Instantaneous Error")
    axes[0].grid()

    axes[1].plot(results["time"], results["integrals"])
    axes[1].set_ylabel("Integral term")
    axes[1].set_title("Integral Accumulation")
    axes[1].grid()

    axes[2].plot(results["time"], results["derivatives"])
    axes[2].set_ylabel("Derivative term")
    axes[2].set_xlabel("Time (s)")
    axes[2].set_title("Derivative Estimate")
    axes[2].grid()

    plt.tight_layout()
    plt.show()

    
results = simulate_pid_control()
plot_pid_terms(results)

### PID tuning

In [ ]:
import ipywidgets as widgets
from ipywidgets import interact, interactive
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

def run_pid_simulation(kp, ki, kd):
    motor = RotaryMotor(
        inertia=0.02,
        friction_viscous=0.05,
        max_torque=1.6,
        friction_coulomb=0.02,
        external_torque=0.01
    )
    encoder = Encoder(motor, noise_std_deg=0.5)
    velocity_filter = ExponentialVelocityFilter(alpha=0.2)

    controller = SpeedControllerPID(
        motor,
        encoder,
        velocity_filter=velocity_filter,
        kp=kp, ki=ki, kd=kd,
        integral_limit=30.0,
        deadband=1.0
    )

    dt = 0.01
    t_max = 10.0
    steps = int(t_max / dt)

    time, ref_vel, real_vel, est_vel, throttle = [], [], [], [], []

    for step in range(steps):
        t = step * dt

        if t < 3.0:
            vel_ref = 30.0 * (t / 3.0)
        elif t < 6.0:
            vel_ref = 30.0
        elif t < 8.0:
            vel_ref = 30.0 * (1 - (t - 6.0) / 2.0)
        else:
            vel_ref = 0.0

        status = controller.update(vel_ref, dt)
        motor.update(dt)

        time.append(t)
        ref_vel.append(vel_ref)
        real_vel.append(motor.get_velocity_deg_s())
        est_vel.append(status["measured_velocity"])
        throttle.append(status["throttle"])

    fig, axes = plt.subplots(2, 1, figsize=(16, 6), sharex=True)

    axes[0].plot(time, ref_vel, label="Setpoint", linestyle='--')
    axes[0].plot(time, real_vel, label="Real velocity")
    axes[0].plot(time, est_vel, label="Estimated velocity", alpha=0.5)
    axes[0].set_ylabel("Velocity (deg/s)")
    axes[0].set_title(f"kp={kp:.2f}, ki={ki:.2f}, kd={kd:.2f}")
    axes[0].legend()
    axes[0].grid()

    axes[1].plot(time, throttle, label="Throttle")
    axes[1].set_xlabel("Time (s)")
    axes[1].set_ylabel("Throttle")
    axes[1].grid()

    plt.tight_layout()
    plt.show()


# ------------------------------
# Sliders interactivos
# ------------------------------
interact(
    run_pid_simulation,
    kp=widgets.FloatSlider(value=0.2, min=0.0, max=1.0, step=0.01),
    ki=widgets.FloatSlider(value=0.03, min=0.0, max=0.1, step=0.001),
    kd=widgets.FloatSlider(value=0.02, min=0.0, max=0.1, step=0.001)
);


## 4. Planificador de trayectoria + PID posiciòn

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import ipywidgets as widgets

def run_position_pid_simulation(kp_pos, ki_pos, kd_pos):
    # Motor y sensores
    motor = RotaryMotor(
        inertia=0.02,
        friction_viscous=0.1,
        max_torque=2.5,
        friction_coulomb=0.12,
        external_torque=0.01
    )
    encoder = Encoder(motor, noise_std_deg=0.5)
    velocity_filter = ExponentialVelocityFilter(alpha=0.2)

    # Controladores
    pos_pid = PositionControllerPID(kp=kp_pos, ki=ki_pos, kd=kd_pos)
    vel_pid = SpeedControllerPID(
        motor,
        encoder,
        velocity_filter=velocity_filter,
        kp=0.01, ki=0.01, kd=0,
        integral_limit=30.0,
        deadband=1.0
    )

    planner = TrapezoidalPlanner(pos0=0, posf=90, vmax=30, amax=20)

    dt = 0.01
    t_max = 10.0
    steps = int(t_max / dt)

    time, pos_ref, pos_real, vel_ref_planner, vel_ref_ctrl, vel_real = [], [], [], [], [], []

    for step in range(steps):
        t = step * dt

        p_ref, v_ref_plan = planner.get_ref(t)
        p_actual = motor.get_position_unwrapped_deg()

        v_ref_ctrl = pos_pid.update(p_ref, p_actual, dt)
        v_ref = np.clip(v_ref_ctrl, -abs(v_ref_plan), abs(v_ref_plan))

        vel_pid.update(v_ref, dt)
        motor.update(dt)

        time.append(t)
        pos_ref.append(p_ref)
        pos_real.append(p_actual)
        vel_ref_planner.append(v_ref_plan)
        vel_ref_ctrl.append(v_ref)
        vel_real.append(motor.get_velocity_deg_s())

    fig, axes = plt.subplots(2, 1, figsize=(16, 6), sharex=True)

    axes[0].plot(time, pos_ref, label="Position Ref")
    axes[0].plot(time, pos_real, label="Position Real")
    axes[0].set_ylabel("Position (deg)")
    axes[0].set_title(f"Position PID: kp={kp_pos:.2f}, ki={ki_pos:.2f}, kd={kd_pos:.2f}")
    axes[0].legend()
    axes[0].grid()

    axes[1].plot(time, vel_ref_planner, label="Planner vel_ref")
    axes[1].plot(time, vel_ref_ctrl, label="Position PID vel_ref")
    axes[1].plot(time, vel_real, label="Velocity Real", alpha=0.7)
    axes[1].set_ylabel("Velocity (deg/s)")
    axes[1].set_xlabel("Time (s)")
    axes[1].grid()
    axes[1].legend()

    plt.tight_layout()
    plt.show()

# ------------------------------
# Sliders interactivos
# ------------------------------
interact(
    run_position_pid_simulation,
    kp_pos=widgets.FloatSlider(value=2.0, min=0.0, max=5.0, step=0.1),
    ki_pos=widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.05),
    kd_pos=widgets.FloatSlider(value=0.0, min=0.0, max=1.0, step=0.05)
);